# Get translated protein product of isoforms

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import gffutils
import pybedtools
import re
import numpy as np

v19db_filename = '/projects/ps-yeolab/genomes/hg19/gencode/v19/gencode.v19.annotation.gtf.db'
v19db = gffutils.FeatureDB(v19db_filename)


/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [2]:
folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper/'.format(folder)


splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

In [3]:
figure_folder = '{}/isoform_protein_properties'.format(alternative_feature_folder)
! mkdir $figure_folder

## Translate isoform

In [4]:
prefix = 'isoform_translations'
translated_fasta = '{}/{}.fa'.format(alternative_feature_folder, prefix)

In [5]:
ll $translated_fasta

-rw-r--r-- 1 obotvinnik 8610952 Mar 29 03:01 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations.fa


In [7]:
!wc -l $translated_fasta

85723 /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations.fa


## Find protein domains using `hmmscan`

Submit a compute cluster job using our in-house tool which writes and submits a bash script

In [8]:
from gscripts.qtools import Submitter

n_processors = 16
pfam_a = '/projects/ps-yeolab/genomes/pfam/release_27/ftp.sanger.ac.uk/pub/databases/Pfam/releases/Pfam27.0/Pfam-A.hmm'
commands = []
# for isoform_number in (1, 2):
hmmscan_out = '{}/{}_hmmscan_pfamA.txt'.format(alternative_feature_folder, prefix)
command = 'hmmscan --domtblout {} --cpu {} --noali --notextw {} {}'.format(hmmscan_out, n_processors, pfam_a, translated_fasta)
commands.append(command)
sub = Submitter(commands, 'hmmscan', walltime='24:00:00', ppn=n_processors, write_and_submit=True,
                err_filename=hmmscan_out+'.err', out_filename=hmmscan_out+'.out')

job ID: 4678196


In [3]:
! qdel 3610818

In [2]:
! cat hmmscan.sh*

#!/bin/bash
#PBS -N hmmscan
#PBS -o /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations_hmmscan_pfamA.txt.out
#PBS -e /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations_hmmscan_pfamA.txt.err
#PBS -V
#PBS -l walltime=24:00:00
#PBS -l nodes=1:ppn=16
#PBS -A yeo-group
#PBS -q home

# Go to the directory from which the script was called
cd $PBS_O_WORKDIR
hmmscan --domtblout /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations_hmmscan_pfamA.txt --cpu 16 --noali --notextw /projects/ps-yeolab/genomes/pfam/release_27/ftp.sanger.ac.uk/pub/databases/Pfam/releases/Pfam27.0/Pfam-A.hmm /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations.fa



In [3]:
! head /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations_hmmscan_pfamA.txt*

==> /projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper//splicing_feature_data/alternative/isoform_translations_hmmscan_pfamA.txt <==
#                                                                                                                                                                                                                                                                         --- full sequence --- -------------- this domain -------------   hmm coord   ali coord   env coord
# target name        accession   tlen query name                                                                                                                                                                                                        accession   qlen   E-value  score  bias   #  of  c-Evalue  i-Evalue  score  bias  from    to  from    to  from    to  acc description of target
#------------------- ---------- -----                                                              